# Демонстрация использования класса `OvervoltageDetector`

Этот ноутбук показывает, как использовать класс `OvervoltageDetector` из `analysis.overvoltage_detector` для анализа осциллограмм на предмет перенапряжений во время однофазных замыканий на землю (ОЗЗ).

Ключевые этапы:
- Загрузка и подготовка данных осциллограммы (`Oscillogram`).
- Нормализация данных (`OscillogramNormalizer`).
- Разделение данных по шинам (`OscillogramToCsvConverter`).
- Поиск зон ОЗЗ и расчет максимального коэффициента перенапряжения в этих зонах.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json
import csv 
import sys

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram
from normalization.normalization import OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter 
from analysis.overvoltage_detector import OvervoltageDetector 

# --- Создание временных директорий ---
base_sample_dir_ov = "temp_sample_data_overvoltage"
source_dir_ov = os.path.join(base_sample_dir_ov, "source_oscillograms_ov")
output_dir_ov = os.path.join(base_sample_dir_ov, "overvoltage_analysis_output")
config_files_dir_ov = os.path.join(base_sample_dir_ov, "config_files_ov")

if os.path.exists(base_sample_dir_ov):
    shutil.rmtree(base_sample_dir_ov)
os.makedirs(source_dir_ov)
os.makedirs(output_dir_ov)
os.makedirs(config_files_dir_ov)

# --- Вспомогательная функция для создания CFG/DAT ---
def create_ov_sample_comtrade(path, name, analog_ch_details, freq=50.0, samp_rate=1000.0, dat_rows=200):
    cfg_content = f"SampleStation,OV_Device,1999\n"
    cfg_content += f"{len(analog_ch_details)},{len(analog_ch_details)}A,0D\n"
    ch_idx = 1
    for ch_name, _ in analog_ch_details:
        # primary/secondary for transformer ratios; a=1, b=0 for raw data output
        cfg_content += f"{ch_idx},{ch_name},A,,V,1.0,0.0,0,-32767,32767,10000,6000,P\n" 
        ch_idx += 1
    cfg_content += f"{freq}\n1\n{samp_rate},{dat_rows}\n"
    cfg_content += "01/01/2023,00:00:00.000000\n01/01/2023,00:00:00.000000\nASCII\n1.0\n"
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f: f.write(cfg_content)
    dat_content = ""
    for i in range(dat_rows):
        dat_content += f"{i+1},{int(i*(1000000/samp_rate))}"
        for _, data_series in analog_ch_details: dat_content += f",{data_series[i % len(data_series)]:.6f}"
        dat_content += "\n"
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f: f.write(dat_content)

# --- Создание файлов осциллограмм ---
time_pts_ov = np.linspace(0, 2*np.pi*10, 200)
samples_per_period_ov = 20
nominal_phase_rms_6kv = 6000.0 / np.sqrt(3)
nominal_phase_peak_6kv = nominal_phase_rms_6kv * np.sqrt(2)

ua_overvoltage = np.concatenate([np.sin(time_pts_ov[:80])*nominal_phase_peak_6kv, 
                                 np.sin(time_pts_ov[80:120])*nominal_phase_peak_6kv*2.5, 
                                 np.sin(time_pts_ov[120:])*nominal_phase_peak_6kv])
un_signal_spef_phys = np.concatenate([np.zeros(80), np.sin(time_pts_ov[80:120])*0.2*nominal_phase_peak_6kv, np.zeros(80)])

create_ov_sample_comtrade(source_dir_ov, "osc_ov_spef", [
    ("U | BusBar-1 | phase: A", ua_overvoltage),
    ("U | BusBar-1 | phase: B", np.sin(time_pts_ov - 2*np.pi/3)*nominal_phase_peak_6kv*0.5),
    ("U | BusBar-1 | phase: C", np.sin(time_pts_ov + 2*np.pi/3)*nominal_phase_peak_6kv*0.5),
    ("U | BusBar-1 | phase: N", un_signal_spef_phys)
])

create_ov_sample_comtrade(source_dir_ov, "osc_ov_normal", [
    ("U | CableLine-1 | phase: A", np.sin(time_pts_ov)*nominal_phase_peak_6kv),
    ("U | CableLine-1 | phase: B", np.sin(time_pts_ov - 2*np.pi/3)*nominal_phase_peak_6kv),
    ("U | CableLine-1 | phase: C", np.sin(time_pts_ov + 2*np.pi/3)*nominal_phase_peak_6kv)
])

ua_sym_sag = np.sin(time_pts_ov)*nominal_phase_peak_6kv*0.7
ub_sym_sag = np.sin(time_pts_ov - 2*np.pi/3)*nominal_phase_peak_6kv*0.7
uc_sym_sag = np.sin(time_pts_ov + 2*np.pi/3)*nominal_phase_peak_6kv*0.7
un_low_u0_phys = np.concatenate([np.zeros(80), np.sin(time_pts_ov[80:120])*0.01*nominal_phase_peak_6kv, np.zeros(80)])
create_ov_sample_comtrade(source_dir_ov, "osc_ov_low_or_sym", [
    ("U | BusBar-1 | phase: A", ua_sym_sag),
    ("U | BusBar-1 | phase: B", ub_sym_sag),
    ("U | BusBar-1 | phase: C", uc_sym_sag),
    ("U | BusBar-1 | phase: N", un_low_u0_phys)
])

print(f"Создана директория с примерами: {source_dir_ov}")

norm_data_ov = [
    ["name", "norm", "1Ub_PS", "1Ub_base", "1Uc_PS", "1Uc_base"],
    ["osc_ov_spef", "YES", "s", nominal_phase_rms_6kv, "s", nominal_phase_rms_6kv],
    ["osc_ov_normal", "YES", "s", nominal_phase_rms_6kv, "s", nominal_phase_rms_6kv],
    ["osc_ov_low_or_sym", "YES", "s", nominal_phase_rms_6kv, "s", nominal_phase_rms_6kv]
]
sample_norm_coef_path_ov = os.path.join(config_files_dir_ov, "sample_norm_coef_ov.csv")
with open(sample_norm_coef_path_ov, "w", newline="", encoding="utf-8") as f: csv.writer(f).writerows(norm_data_ov)

analog_names_path_ov = os.path.join(config_files_dir_ov, "dict_analog_names_ov.json")
analog_content_ov = { "bus1": { 
    "U_BB_A": ["U | BusBar-1 | phase: A"], "U_BB_B": ["U | BusBar-1 | phase: B"],
    "U_BB_C": ["U | BusBar-1 | phase: C"], "U_BB_N": ["U | BusBar-1 | phase: N"],
    "U_CL_A": ["U | CableLine-1 | phase: A"], "U_CL_B": ["U | CableLine-1 | phase: B"],
    "U_CL_C": ["U | CableLine-1 | phase: C"], "U_CL_N": ["U | CableLine-1 | phase: N"]
}} 
with open(analog_names_path_ov, "w", encoding="utf-8") as f: json.dump(analog_content_ov, f)
discrete_names_path_ov = os.path.join(config_files_dir_ov, "dict_discrete_names_ov.json")
with open(discrete_names_path_ov, "w", encoding="utf-8") as f: json.dump({}, f) 

print("Созданы конфигурационные файлы для демонстрации.")

## 1. Инициализация компонентов

In [ ]:
ov_config = {
    'VALID_NOMINAL_VOLTAGES': {round(nominal_phase_rms_6kv, 2)}, 
    'SPEF_THRESHOLD_U0': 0.05,  # PU (normalized U0 > 0.05 PU of phase nominal)
    'SPEF_THRESHOLD_Un': 0.03, # PU (normalized Un > 0.03 PU of phase nominal)
    'SPEF_MIN_DURATION_PERIODS': 2, 
    'SIMILAR_AMPLITUDES_FILTER_ENABLED': True,
    'SIMILAR_AMPLITUDES_MAX_RELATIVE_DIFFERENCE': 0.15, 
    'verbose': True, 
    'norm_yes_phrase': 'YES'
}

normalizer_ov = OscillogramNormalizer(norm_coef_file_path=sample_norm_coef_path_ov, is_print_message=True)

bus_splitter_ov = OscillogramToCsvConverter(
    normalizer=normalizer_ov, 
    raw_path=source_dir_ov, 
    csv_path=output_dir_ov,
    dict_analog_names_path=analog_names_path_ov,
    dict_discrete_names_path=discrete_names_path_ov,
    is_print_message=False # Reduce noise from this component for this demo
)

norm_coeffs_df_ov = pd.read_csv(sample_norm_coef_path_ov)
detector = OvervoltageDetector(
    config=ov_config,
    normalizer=normalizer_ov,
    bus_splitter=bus_splitter_ov,
    norm_coef_df=norm_coeffs_df_ov 
)
print("\nOvervoltageDetector и зависимости инициализированы.")

## 2. Анализ директории на перенапряжения при ОЗЗ (`analyze_directory`)

In [ ]:
output_ov_report_csv = os.path.join(output_dir_ov, "overvoltage_report.csv")
error_log_ov_path = os.path.join(output_dir_ov, "overvoltage_errors.log")

print(f"\nЗапуск анализа директории: {source_dir_ov}")
detector.analyze_directory(
    osc_folder_path=source_dir_ov,
    output_csv_path=output_ov_report_csv,
    log_path=error_log_ov_path
)

if os.path.exists(output_ov_report_csv):
    print(f"\nСодержимое отчета о перенапряжениях ({output_ov_report_csv}):")
    try: 
        from IPython.display import display
        report_df = pd.read_csv(output_ov_report_csv)
        display(report_df)
    except ImportError:
        print(pd.read_csv(output_ov_report_csv))
    except Exception as e:
        print(f"Ошибка чтения CSV отчета: {e}")
else:
    print(f"Отчет о перенапряжениях не был создан.")

if os.path.exists(error_log_ov_path) and os.path.getsize(error_log_ov_path) > 0:
    print(f"\nСодержимое лога ошибок ({error_log_ov_path}):")
    with open(error_log_ov_path, "r", encoding='utf-8') as f: print(f.read())
else:
    print(f"\nЛог ошибок ({error_log_ov_path}) пуст или не создан.")

## 3. Демонстрация `analyze_oscillogram` (для одного файла)

In [ ]:
try:
    osc_example_ov_path = os.path.join(source_dir_ov, "osc_ov_spef.cfg")
    osc_obj_ov = Oscillogram(osc_example_ov_path)
    print(f"\nАнализ одного файла: {osc_obj_ov.file_hash}")
    
    # detector.verbose = True # Already set in config for this demo
    result_single = detector.analyze_oscillogram(osc_obj_ov)

    if result_single:
        print("Результат анализа для одного файла:")
        print(result_single)
    else:
        print("Для данного файла значимых перенапряжений при ОЗЗ не найдено или файл не прошел проверки.")
except Exception as e:
    print(f"Ошибка при анализе одного файла: {e}")

## 4. Демонстрация `copy_spef_oscillograms` (статический метод)

In [ ]:
copied_ov_dir = os.path.join(output_dir_ov, "copied_overvoltage_oscillograms")

if os.path.exists(output_ov_report_csv): 
    print(f"\nКопирование осциллограмм из отчета {output_ov_report_csv} в {copied_ov_dir}...")
    OvervoltageDetector.copy_spef_oscillograms(
        report_csv_path=output_ov_report_csv,
        source_osc_folder_path=source_dir_ov, 
        destination_folder_path=copied_ov_dir,
        is_print_message=True
    )
    
    if os.path.exists(copied_ov_dir):
        print("\nСодержимое директории со скопированными осциллограммами:")
        for item in os.listdir(copied_ov_dir): print(f"- {item}")
else:
    print(f"Отчет {output_ov_report_csv} не найден, копирование не будет выполнено.")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir_ov):
        shutil.rmtree(base_sample_dir_ov)
        print(f"\nВременная директория {base_sample_dir_ov} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir_ov}: {e}")